In [35]:
#import nltk
import re

In [2]:
# загрузим привычный корпус
corpus = nltk.corpus.conll2000

In [3]:
# возьмем список (неразмеченных) слов и предложений из корпуса
words = corpus.words()
sents = corpus.sents()


['Confidence', 'in', 'the', 'pound', 'is']

NLTK умеет считать частоты по корпусу и даже извлекать триграммы из списков слов:

In [4]:
from nltk.probability import FreqDist
from nltk import trigrams

unigram_fd = FreqDist()
words_fd = FreqDist()
sents_fd = FreqDist()

unigram_fd.update(words)
words_fd.update(trigrams(words))
for sent in sents:
    sents_fd.update(trigrams(sent))

print('Length of unigram FreqDist = {}'.format(len(unigram_fd)))
print('Length of trigrams FreqDist by words = {}'.format(len(words_fd)))
print('Length of trigrams FreqDist by sents = {}'.format(len(sents_fd)))

Length of unigram FreqDist = 21589
Length of trigrams FreqDist by words = 212933
Length of trigrams FreqDist by sents = 196878


In [ ]:
# Можно почитать, что представляет собой объект FreqDist
?words_fd
# или: help(words_fd)

Попробуем вывести самые частые 10 триграмм:

In [5]:
sents_fd.most_common(10)

[(('million', ',', 'or'), 256),
 (('a', 'share', ','), 240),
 ((',', "''", 'says'), 195),
 (('cents', 'a', 'share'), 161),
 ((',', "''", 'said'), 138),
 (('%', 'to', '$'), 134),
 ((',', 'or', '$'), 125),
 (('the', 'company', "'s"), 114),
 ((',', "''", 'he'), 109),
 (('a', 'year', 'earlier'), 91)]

Всевозможные улучшения для языковых моделей в NLTK тоже есть, и тоже не очень удобные:

In [6]:
from nltk.probability import LaplaceProbDist, KneserNeyProbDist

eval_trigrams = [
    ('who', 'are', 'not'),
    ('who', 'is', 'the'),
    ('I', 'love', 'you'),
    ('in', 'San', 'Francisco'),
    ('to', 'San', 'Diego'),
]

laplace = LaplaceProbDist(sents_fd)
kn = KneserNeyProbDist(sents_fd)
for t in eval_trigrams:
    print(t, laplace.prob(t), kn.prob(t))

('who', 'are', 'not') 2.3036219848467746e-06 0.009112681815526607
('who', 'is', 'the') 4.607243969693549e-06 0.011904761904761904
('I', 'love', 'you') 2.3036219848467746e-06 0.0
('in', 'San', 'Francisco') 4.146519572724194e-05 0.7386363636363636
('to', 'San', 'Diego') 2.3036219848467746e-06 0.012499999999999999


Как проверить, какие из триграмм были в частотном списке?

In [9]:
sents_fd[('who', 'are', 'not')]

0

## Генерация случайных текстов (character-based)

Для того, чтобы сгенерировать случайный текст нужно запастись двумя вещами:

1. Обучающий корпус.
2. Языковая модель.

In [61]:
# 1. Читаем обучающий корпус из input.txt
with open('messages.txt', encoding = 'utf-8') as f:
    
   corpus = f.read()


reg_dima = re.compile(r'Дмитрий Залманов :\n(.*?)\n')
reg_ya = re.compile(r'Наташа Озерчук :\n(.*?)\n')
msg_d = re.findall(reg_dima, corpus)
msg_dima = ' '.join(msg_d)
msg_y = re.findall(reg_ya, corpus)
msg_ya = ' '.join(msg_y)

2.Как обычно - реализуем класс для языковой модели.

In [53]:
from collections import Counter, defaultdict

class CharLM:
    def __init__(self, data, order=4):
        self.order = order
        self.ngrams = defaultdict(Counter)
        pad = '~' * order  # специальный символ для начала предложения (для первого символа будет ~~)
        data = pad + data
        # Для каждой n-граммы из символов посчитаем символы, которые идут перед ней
        # Например, если порядок модели 2, а корпус выглядит так 'abcbcb':
        # self.ngrams['~~']['a'] == 1
        # self.ngrams['~a']['b'] == 1
        # self.ngrams['ab']['c'] == 1
        # self.ngrams['bc']['b'] == 2
        # self.ngrams['cb']['c'] == 1
        for i in range(len(data) - order):
            history, char = data[i:i+order], data[i+order]
            self.ngrams[history][char] +=1
        self.lm = {history: self.normalize(chars) for history, chars in self.ngrams.items()}
    
    def normalize(self, counter):
        # Всё как обычно - превращаем частоты в вероятности
        # сделаем только в одну строчку - more pythonic ;)
        sum_ = sum(counter.values())
        return [(char, count / sum_) for char, count in counter.items()]
    
    def __getitem__(self, history):
        return self.lm[history]

Обучаем модель:

In [54]:
lm = CharLM(msg_dima, order=2)

Посмотрим, что получилось:

In [55]:
lm['ну']

KeyError: 'ну'

In [18]:
lm['of']

[(' ', 0.8530393325387365),
 ('t', 0.025387365911799763),
 ('f', 0.06930870083432658),
 (',', 0.004886769964243146),
 ("'", 0.0005363528009535161),
 ('e', 0.004350417163289631),
 ('\n', 0.02234803337306317),
 ('-', 0.0004171632896305125),
 ('a', 0.0017282479141835518),
 (';', 0.0017878426698450535),
 ('o', 0.0012514898688915374),
 (':', 0.0010727056019070322),
 ('u', 0.0015494636471990466),
 ('.', 0.005184743742550656),
 ('!', 0.0004171632896305125),
 ('s', 0.0012514898688915374),
 ('i', 0.003933253873659118),
 ('?', 0.0015494636471990466)]

А теперь напишем функцию для генерации случайных текстов!

In [51]:
from random import random

def generate_letter(lm, history):
    history = history[-lm.order:]
    # По предыдущим символам будем генерировать следующий с учётом вероятностей
    dist = lm[history]
    x = random()
    for char, prob in dist:
        x = x - prob
        if x <= 0:
            return char
        
def generate_text(lm, n_letters=1000):
    history = '~' * lm.order
    out = []
    # Генерируем текст длины n_letters
    for i in range(n_letters):
        # на каждом шаге генерируем новый символ
        c = generate_letter(lm, history)
        # обновляем историю и результат
        history = history[-lm.order:] + c
        out.append(c)
    return ''.join(out)

Попробуем генерировать тексты разной длины с помощью моделей разного порядка -- 2...10?

Какой результат более разнообразный? Какой более связный?

In [52]:
print(generate_text(lm, 1000))

TypeError: 'CharLM' object is not subscriptable

In [68]:
lm8 = CharLM(msg_ya, order=8)
print(generate_text(lm8, 80))

Ой они классные тоже, я на них была! ну это бывает я не верю))  я помню больше ч


In [42]:
len(msg_dima)

1170522

In [43]:
len(msg_ya)

1090330

In [62]:
msg_dima = msg_dima[:100000]
msg_ya = msg_ya[:100000]

In [60]:
print(msg_dima[:1000])

Хоспаде, ты шикарна, решил чекнуть твои аудио, нашел Тролль Гнет Ельда во что я только не играю:D Но в лол больше всего, наверное) А ты тоже играешь или просто знаешь?)нуууу, не прям фанат, но очень люблю некоторые песнихотел на концерт сходить когда-нибудьага, там атмосфера милаяУчеба все время занимает?)или еще что-то?а ты из общаги ездишь?и где твой корпус, кстати?хмммм, сколько дорога занимает?жизненно:(единственный негатив от учебы)нес Бабушкинской езжув Строгиноот моего дома до любого нормального универа ехать больше часа:D:D	от моего дома до любого нормального универа ехать больше часа:Dхмммма где корпус то?На Басманной шоль?блин:Dну тут анлакия еще в 3 классе понял, что я не гуманитарий ни разуа, лол, он у вас теперь?ну не, физика это клево, конечноно четнет, спасибо:D	я еще в 3 классе понял, что я не гуманитарий ни разуhttps://vk.com/patrick2018ХМММММММММрешил загуглить концерт Тролль Гнет Ельнуууу, сначала я получил 3 по рисованию. в 3(!) классе. Потом понял, что не умею писа

In [71]:
print(generate_text(lm8, 80))

Ой они классные стикеры с котом 	которые шышки едят 	который с трудом проводит р
